In [187]:
import numpy as np

In [188]:
# basic parameters:
arrival_rate = 0.8
background_speed = 12 # 25mph = 11.176 m/s
cruising_speed = 0.25 * background_speed
segment_length = 1000
vehicle_length = 4.5 # a typical vehicle length is 4.5 meters
pull_over_dist = 3 # The artifical pull over distance is 3 m


# Generate possion arrival for left land and right lane as the inter-arrival time is following exponential distribution
beta = 1 / arrival_rate
left_interarr_times = np.random.exponential(scale = beta, size = 1000)
right_interarr_times = np.random.exponential(scale = beta, size = 1000)

# So the inter-vehicle distances could be calculated as time * background_speed
left_interarr_dists = left_interarr_times * background_speed
right_interarr_dists = right_interarr_times * background_speed

# The physical stamp is the distance between of the fronts of two vehicle
# To make sure that there is at least a vehicle, we need to filter those distance < 1.1 * vehicle_length and replace with 1.1 * vehicle_length
left_interarr_dists_modified = np.clip(left_interarr_dists, 1.1 * vehicle_length, np.inf)
right_interarr_dists_modified = np.clip(right_interarr_dists, 1.1 * vehicle_length, np.inf)

# Get the coordinates from the intersection of the front of each vehicles, which is the cumulative of distances between vehicles:
left_vehicle_coordinates = np.cumsum(left_interarr_dists_modified, dtype = float)
left_vehicle_coordinates = left_vehicle_coordinates[left_vehicle_coordinates < segment_length]
right_vehicle_coordinates = np.cumsum(right_interarr_dists_modified, dtype = float)
right_vehicle_coordinates = right_vehicle_coordinates[right_vehicle_coordinates < segment_length]

In [189]:
# required length for pull over slot:
required_length = 2 * vehicle_length

In [190]:
# Lets look again at the left vehicle coordinates of the vehicle on the left lane:
left_vehicle_coordinates

array([ 17.28818613,  42.61618536,  50.91567539,  93.2622698 ,
       105.27796735, 153.10271201, 207.18402114, 231.01080523,
       245.67111759, 253.7242504 , 261.73671456, 314.41347323,
       319.36347323, 324.31347323, 352.01033359, 385.79281244,
       393.32687587, 398.27687587, 403.22687587, 439.2220146 ,
       445.99580578, 456.73328596, 464.31238325, 472.35710356,
       477.30710356, 482.25710356, 492.51699549, 497.46699549,
       506.88421203, 511.83421203, 564.56211535, 613.10017169,
       621.2254532 , 626.1754532 , 631.1254532 , 636.0754532 ,
       641.0254532 , 675.31433676, 682.50859387, 687.45859387,
       694.86613662, 709.27857797, 751.98103121, 771.83412336,
       777.202149  , 786.60169072, 803.4151497 , 808.3651497 ,
       813.3151497 , 826.77802571, 860.11031312, 874.70263431,
       879.65263431, 886.03607605, 891.50026366, 896.45026366,
       901.57456538, 926.9844789 , 950.84322735, 967.50790218,
       995.4164654 ])

In [191]:
# And we can get the number of the vehicle in front of Emergency Vehicle of interest:
left_vehicle_coordinates.shape

(61,)

In [192]:
# Lets look at the coordinates of the vehicles on the right lane:
right_vehicle_coordinates

array([ 15.05355146,  20.00355146,  34.56377406,  39.51377406,
        47.7477869 ,  52.6977869 ,  73.33684786,  78.28684786,
        90.20665198,  95.15665198, 110.6692602 , 115.6192602 ,
       123.89437131, 153.27613661, 158.22613661, 201.20375951,
       210.68086435, 229.82180779, 234.77180779, 245.15101574,
       250.98851355, 264.80674074, 278.89601377, 304.83018065,
       315.65908292, 326.82375733, 335.5489024 , 348.46343075,
       353.41343075, 358.36343075, 382.90082678, 399.81370712,
       423.56924147, 429.7916383 , 439.54274097, 484.62170952,
       504.01104946, 509.66547288, 527.08082481, 534.76166435,
       544.41729028, 549.36729028, 554.48286628, 562.87919002,
       584.27120673, 589.22120673, 594.17120673, 649.95489294,
       654.90489294, 671.65662954, 676.60662954, 695.22401338,
       707.28677008, 712.23677008, 737.83689141, 758.71137194,
       773.84521122, 782.12584624, 805.29918376, 858.63674947,
       863.58674947, 886.96212934, 904.83984961, 909.78

In [193]:
# The number of vehicles on the right lane is:
len(right_vehicle_coordinates)

70

In [208]:
# We then have to calculate the exact coordinates of those slots, starting from 0:
def get_empty_slots(right_vehicle_coordinates):
    empty_slots = []
    current_position = 0
    for i in range(len(right_vehicle_coordinates)):
        head = current_position
        tail = right_vehicle_coordinates[i]
        empty_slots.append([head, tail])
        current_position = tail + vehicle_length
    if (current_position - vehicle_length) < segment_length :
        empty_slots.append([current_position, segment_length])
    return empty_slots

In [209]:
# And we could know that there should be number_vehicle + 1 slots for possible pulling over:
len(get_empty_slots(right_vehicle_coordinates))

71

In [196]:
# Pull over rules for distributed systems:
# 1. a vehicle could only pull over to a spot that is physically ahead of him, which means the head of this slot
# should be smaller than the front of the vehicle;
# 2. the empty slot should at least be 2 * vehicle length long
# 3. Since there is a pull over distance, the back of the vehicle - pull over distance should be smaller than 
# the tail of the slot
def can_pull_over(vehicle_coordinate, slot):
    head = slot[0]
    tail = slot[1]
    vehicle_back = vehicle_coordinate + vehicle_length
    if (tail - head) < required_length:
        return False
    elif vehicle_coordinate - pull_over_dist < head:
        return False
    return True

In [197]:
# Then we can see which slots a vehicle can pull over into, if there is no available slots, then it will exit from the intersection
# For example, lets take a look at the first vehicle:
first_veh_coordinate = left_vehicle_coordinates[0]
first_veh_coordinate

17.28818613365445

In [198]:
for slot_element in empty_slots:
    result = can_pull_over(first_veh_coordinate, slot_element)
    if result:
        print(slot_element)
    else:
        break

[0, 15.053551462985336]


In [199]:
# A special rule regarding the distributed system is that all vehicles will try to pull over into their nearest
# slot. However, with respect to time, we might not know which vehicle would pull in first.
# So, we need to calculate that and pick the smallest one to be timestamp of interest. Through the process, vehicles
# to be cleared and empty slots are changing per timestamp of interest

In [200]:
# First of all, we need to know which slot each vehicle is going to pull over into at timestamp 0:
def slot_to_park_distributed(vehicle_coordinate, empty_slots):
    # default is exiting at the intersection
    res = 0
    
    for slot_element in empty_slots:
        if can_pull_over(vehicle_coordinate, slot_element):
            res = slot_element     
    return res

In [201]:
# Check if all satisfy the pull over slots:
for vehicle_coordinate in left_vehicle_coordinates:
    print(vehicle_coordinate, slot_to_park_distributed(vehicle_coordinate, empty_slots))

17.28818613365445 [0, 15.053551462985336]
42.61618535837399 [24.503551462985335, 34.563774057132974]
50.91567539245367 [24.503551462985335, 34.563774057132974]
93.26226980485953 [57.19778689744099, 73.33684785893567]
105.27796735113738 [99.65665197873678, 110.66926020340097]
153.10271201257504 [128.3943713116136, 153.27613660911774]
207.1840211361698 [162.72613660911773, 201.20375950859483]
231.01080523023185 [215.18086435361806, 229.82180779019868]
245.67111758889314 [215.18086435361806, 229.82180779019868]
253.7242503981516 [215.18086435361806, 229.82180779019868]
261.7367145645494 [255.48851354849992, 264.8067407396845]
314.41347323185437 [283.3960137673763, 304.8301806501367]
319.36347323185436 [283.3960137673763, 304.8301806501367]
324.31347323185435 [283.3960137673763, 304.8301806501367]
352.0103335940871 [283.3960137673763, 304.8301806501367]
385.7928124419832 [362.8634307453442, 382.9008267848511]
393.3268758740754 [387.4008267848511, 399.8137071192003]
398.2768758740754 [387.4

In [202]:
# Compute the smallest pull over time at certain timestamp:
def next_timestamp(vehicle_coordinates, empty_slots):
    pull_over_times = []
    for vehicle in vehicle_coordinates:
        
        vehicle_back = vehicle + vehicle_length
        tail = slot_to_park_distributed(vehicle, empty_slots)
        if tail != 0:
            tail = tail[1]
            
        
        if tail <= vehicle_back - pull_over_dist:
            pull_over_time = (vehicle_back - tail) / cruising_speed
        else:
            pull_over_time = pull_over_dist / cruising_speed
        pull_over_times.append(pull_over_time)
    
    min_timestamp = np.min(pull_over_times)
    min_positions = list([i for i, x in enumerate(pull_over_times) if x == min_timestamp])
    return min_timestamp, min_positions

In [240]:
# Update the available slots
def update(vehicle_coordinates, min_positions, timestamp, right_vehicle_coordinates):
    # Get the coordinates of the vehicle to pull over
    vehicles_to_pull_over = vehicle_coordinates[min_positions]
    # picture the new coordinates of these vehicles including the old ones by timestamp
    new_coordinates = vehicles_to_pull_over - timestamp * cruising_speed
    # print(new_coordinates)
    # print(right_vehicle_coordinates)
    right_vehicle_coordinates = np.append(right_vehicle_coordinates, new_coordinates)
    right_vehicle_coordinates = np.sort(right_vehicle_coordinates)
    right_vehicle_coordinates = np.clip(right_vehicle_coordinates, 0, np.inf)
    
    # Compute the new available empty slots
    new_empty_slots = get_empty_slots(right_vehicle_coordinates)
    # update the vehicle coordinates: decrement the position and get rid of the pulled over:
    new_vehicle_coordinates = vehicle_coordinates - timestamp * cruising_speed
    new_vehicle_coordinates = np.delete(new_vehicle_coordinates, min_positions)
    new_vehicle_coordinates = np.clip(new_vehicle_coordinates, 0, np.inf)
    
    return new_vehicle_coordinates, new_empty_slots, right_vehicle_coordinates

In [241]:
def timing_process(left_vehicle_coordinates, right_vehicle_coordinates):
    time_counter = 0
    while left_vehicle_coordinates != []:
        print(len(left_vehicle_coordinates))
        # Compute the empty slots on the right lane:
        current_empty_slots = get_empty_slots(right_vehicle_coordinates)
        
        # Calculate the timestamp of the next event:
        min_timestamp, min_positions = next_timestamp(left_vehicle_coordinates, current_empty_slots)
        
        time_counter += min_timestamp
        
        # update:
        left_vehicle_coordinates, empty_slots, right_vehicle_coordinates = update(left_vehicle_coordinates, min_positions, min_timestamp, right_vehicle_coordinates)
        
    return time_counter

In [242]:
test1 = left_vehicle_coordinates
test2 = right_vehicle_coordinates

In [243]:
timing_process(test1, test2)

61
32
29
28
26
25
24
23
20
19
18
17
14
13
12
11
10
9
8
7
6
5
4
3
2


/Users/suhaoran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: elementwise != comparison failed; this will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/suhaoran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  This is separate from the ipykernel package so we can avoid doing imports until


109.0258070334489